In [ ]:
# %pip install opencv-python
# %pip install pyautogui pydirectinput opencv-python Pillow
# %pip install fastai
# %pip install torch torchvision torchaudio
# %pip install opencv-python
# %pip install pydirectinput
# %pip install timm

In [ ]:
from fastai.vision.all import *
import pathlib
import numpy as np
import cv2
from PIL import ImageGrab, Image
import pydirectinput
import time
import pathlib
import fastai
import torch
 
# Fix for path issues between different systems (Linux -> Windows)
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
 
# Laad het FastAI model
learn = load_learner('ad-model5.pkl')
 
# Restore PosixPath to its original state (if necessary)
pathlib.PosixPath = temp
 
# Laad het FastAI model
#learn = load_learner('ad-model.pkl')
 
# Stel de afbeeldingsgrootte in zoals vereist door het model
img_height = 180
img_width = 180
 
# Haal de verschillende klassen op
data_dir = pathlib.Path("../Screenshots")  # Stel het pad in naar de mappen met data
dls = learn.dls
class_names = dls.vocab
print(class_names)
 
# Functie voor autonoom rijden
def ad():
    screen_width = 1920  # Pas dit aan naar je werkelijke schermresolutie
    screen_height = 1080  # Pas dit aan naar je werkelijke schermresolutie
    # left_half_bbox = (0, 0, screen_width // 2, screen_height)  # Alleen de linker helft van het scherm
 
    while(True):
        # Maak een screenshot van de linker helft van het scherm
        # printscreen = np.array(ImageGrab.grab(bbox=left_half_bbox))  
        printscreen = np.array(ImageGrab.grab())  
 
        # Bewerken van afbeelding en voorspelling maken
        processed_img = cv2.cvtColor(printscreen, cv2.COLOR_RGB2GRAY)  # Converteer naar grijswaarden
        processed_img = cv2.Canny(processed_img, threshold1=200, threshold2=300)  # Gebruik Canny edge detection
        cv2.imshow('window', processed_img)  # Toon de verwerkte afbeelding
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2RGB)  # Converteer terug naar RGB voor modelinvoer
        processed_img = Image.fromarray(processed_img)  # Converteer naar PIL afbeelding
        processed_img = processed_img.resize((img_width, img_height))  # Schaal naar vereiste afbeeldingsgrootte
 
        # Maak voorspelling met FastAI model
        prediction, _, probs = learn.predict(processed_img)
        direction = str(prediction)  # Haal de voorspelde richting op
        print(f'Predicted direction: {direction}')
 
        # Toetsen indrukken afhankelijk van de voorspelling (op basis van QWERTY)
        if direction == 'd':  # Rechts
            pydirectinput.keyDown('d')  # 'd' blijft hetzelfde voor rechts (QWERTY of AZERTY)
            pydirectinput.keyDown('w')  # Vooruit is 'w' in QWERTY (hoewel het 'z' is op AZERTY)
            time.sleep(0.1)  # Pas de duur aan voor gewenste snelheid
            pydirectinput.keyUp('d')  # Toets loslaten
            pydirectinput.keyUp('w')
        elif direction == 'q':  # Links
            pydirectinput.keyDown('a')  # Links is 'a' in QWERTY (hoewel het 'q' is op AZERTY)
            pydirectinput.keyDown('w')  # Vooruit met 'w' in QWERTY
            time.sleep(0.1)  # Pas de duur aan voor gewenste snelheid
            pydirectinput.keyUp('a')  # Toets loslaten
            pydirectinput.keyUp('w')
        elif direction == 'z':  # Rechtdoor
            pydirectinput.keyDown('w')  # Vooruit is 'w' in QWERTY (hoewel het 'z' is op AZERTY)
            time.sleep(0.1)
            pydirectinput.keyUp('w')  # Toets loslaten
 
        # Stop als de 'q'-toets wordt ingedrukt
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
 
# Start autonoom rijden
ad()

: 